In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from scipy.stats import zscore

data = pd.read_csv("data/train.csv")
data = data.dropna()

data

2024-02-06 19:31:04.131083: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 19:31:04.155232: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 19:31:04.155262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 19:31:04.156013: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 19:31:04.160501: I tensorflow/core/platform/cpu_feature_guar

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
trainsplit = 0.6
s = data.shape[0]
xdata = data.iloc[:,1:]
ydata = data.iloc[:,0]

# xdata = xdata.apply(zscore)
# xdata = xdata.fillna(0)

xtrain = xdata.iloc[:int(trainsplit*s),:]
ytrain = ydata[:int(trainsplit*s)]

xtest = xdata.iloc[int(trainsplit*s):]
ytest = ydata[int(trainsplit*s):]

In [3]:
xtrain

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
ytrain

0        1
1        0
2        1
3        4
4        0
        ..
25195    3
25196    0
25197    7
25198    7
25199    3
Name: label, Length: 25200, dtype: int64

In [5]:
model = tf.keras.Sequential([
    layers.LayerNormalization(),
    # layers.Conv1D(784, 3),
    layers.Dense(784,activation='relu', kernel_initializer=tf.keras.initializers.HeNormal()),
    # layers.Dense(184, activation = 'relu', kernel_initializer = tf.keras.initializers.HeNormal()),
    # layers.Dense(46, activation = 'relu', kernel_initializer = tf.keras.initializers.HeNormal()),
    layers.Dense(10,activation='softmax')
])

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(xtrain, ytrain, epochs=10)

Epoch 1/10
788/788 [==============================] - 6s 7ms/step - loss: 0.2462 - accuracy: 0.9255
Epoch 2/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0898 - accuracy: 0.9712
Epoch 3/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0563 - accuracy: 0.9817
Epoch 4/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0399 - accuracy: 0.9875
Epoch 5/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0326 - accuracy: 0.9890
Epoch 6/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0266 - accuracy: 0.9915
Epoch 7/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0206 - accuracy: 0.9940
Epoch 8/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0251 - accuracy: 0.9925
Epoch 9/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0221 - accuracy: 0.9929
Epoch 10/10
788/788 [==============================] - 6s 7ms/step - loss: 0.0145 - accuracy: 0.9956

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_normalization (Layer  (None, 784)               1568      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 784)               615440    
                                                                 
 dense_1 (Dense)             (None, 10)                7850      
                                                                 
Total params: 624858 (2.38 MB)
Trainable params: 624858 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
results = model.evaluate(xtrain, ytrain)
print(results)

788/788 [==============================] - 2s 2ms/step - loss: 0.0122 - accuracy: 0.9963
[0.012199838645756245, 0.9962698221206665]


In [7]:
results = model.evaluate(xtest, ytest)
print(results)

525/525 [==============================] - 1s 2ms/step - loss: 0.1655 - accuracy: 0.9720
[0.16553957760334015, 0.9720237851142883]


In [8]:
testdata = pd.read_csv("data/test.csv")
testdata

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data = data.dropna()
data = data.apply(zscore)
data = data.fillna(0)
xtest = data

In [10]:
xtest

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,-1.197025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034737,-0.02527,-0.018026,-0.011473,-0.009099,-0.006897,0.0,0.0,0.0,0.0
1,-1.543321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034737,-0.02527,-0.018026,-0.011473,-0.009099,-0.006897,0.0,0.0,0.0,0.0
2,-1.197025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034737,-0.02527,-0.018026,-0.011473,-0.009099,-0.006897,0.0,0.0,0.0,0.0
3,-0.158134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034737,-0.02527,-0.018026,-0.011473,-0.009099,-0.006897,0.0,0.0,0.0,0.0
4,-1.543321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034737,-0.02527,-0.018026,-0.011473,-0.009099,-0.006897,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,-1.543321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034737,-0.02527,-0.018026,-0.011473,-0.009099,-0.006897,0.0,0.0,0.0,0.0
41996,-1.197025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034737,-0.02527,-0.018026,-0.011473,-0.009099,-0.006897,0.0,0.0,0.0,0.0
41997,0.880757,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034737,-0.02527,-0.018026,-0.011473,-0.009099,-0.006897,0.0,0.0,0.0,0.0
41998,0.534460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034737,-0.02527,-0.018026,-0.011473,-0.009099,-0.006897,0.0,0.0,0.0,0.0


In [11]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision
# import numpy as np
# import pandas as pd


# data = pd.read_csv("data/train.csv")
# x_data = torch.tensor(data.iloc[:,1:].values)
# y_data = torch.tensor(data.iloc[:,0].values)


# class MyModel(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.hl1 = nn.Linear(50, 784)
#         self.hl2 = nn.Linear(25, 50)
#         self.hl3 = nn.Linear(1, 25)
        
#     def forward(self, x):
#         x = self.hl1(x)
#         x = self.hl2(x)
#         x = self.hl3(x)
#         return x

# model = MyModel()
# loss_fn = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.8)

# n_epochs = 1000
# for epoch in range(n_epochs):
#     # forward, backward, and then weight update
#     y_pred = model(x_data)
#     loss = loss_fn(y_pred, y_data)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()

#     print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))

# test_data = pd.read_csv("data/test.csv")
# final = test_data.shape[0]
# correct = (predicted == target_tensor).sum()
# print('- Accuracy: %f %%' % (100 * float(correct) / final))
# print('- CE : %2.2f, KL : %2.2f' % (cross_entropy.item(), kl.item()))
